In [5]:
import pandas as pd
results = pd.DataFrame([[34508, 1e-4, 1e-4, 1e-3, 24, 0.47440, 0.35912, 0.40879, 1236, 30258, 0.225215599, 0.065768525],
                        [34510, 1e-4, 1e-4, 1e-5, 64, 0.46884, 0.36772, 0.41217, 1486, 74339, 0.271612436, 0.0616837405],
                        [34511, 1e-4, 1e-4, 1e-6, 75, 0.46578, 0.37260, 0.41401, 3986, 148216, 0.30142808, 0.0620658249],
                        [34512, 1e-4, 1e-4, 1e-7, 24, 0.46324, 0.36765, 0.40995, 18028, 6730523, 0.408272833, 0.0734235644],
                        [34513, 0., 1e-4, 1e-5, 37, 0.46248, 0.36882, 0.41037, 40333, 9413252, 0, 4.11571836],
                        [34514, 1e-4, 0., 1e-5, 60, 0.47707, 0.38251, 0.42459, 1434, 73891, 0.316947281, 0]],
                        columns=['jobID', 'lasso', 'group_lasso', 'threshold', 'epochs', 'Precision', 'recall', 'F1', 'NODES_VOCAB nonzeros', 'SUBTOKEN_VOCAB nonzeros', 'Lasso Reg', 'Group Lasso Reg'])

In [3]:
def get_logs(job, epoch):
    OUTPUT_FOLDER = './SLURM'
    
    
    
    with open(OUTPUT_FOLDER + f'{job}.out','r') as f:
        line = f.readline()
        while line != 'i'
        
    with open(OUTPUT_FOLDER + f'errors_{job}','r') as f:
        errors_file = f.read()




In [6]:
results

,jobID,lasso,group_lasso,threshold,epochs,Precision,recall,F1,NODES_VOCAB nonzeros,SUBTOKEN_VOCAB nonzeros,Lasso Reg,Group Lasso Reg
0,34508,0.0001,0.0001,1.000000e-03,24,0.47440,0.35912,0.40879,1236,30258,0.225216,0.065769
1,34510,0.0001,0.0001,1.000000e-05,64,0.46884,0.36772,0.41217,1486,74339,0.271612,0.061684
2,34511,0.0001,0.0001,1.000000e-06,75,0.46578,0.37260,0.41401,3986,148216,0.301428,0.062066
3,34512,0.0001,0.0001,1.000000e-07,24,0.46324,0.36765,0.40995,18028,6730523,0.408273,0.073424
4,34513,0.0000,0.0001,1.000000e-05,37,0.46248,0.36882,0.41037,40333,9413252,0.000000,4.115718
5,34514,0.0001,0.0000,1.000000e-05,60,0.47707,0.38251,0.42459,1434,73891,0.316947,0.000000


In [ ]:
python3 code2seq.py --load models/java-large-model/model_iter60.release --test data/java-small/java-small.test.c2s